In [1]:
import pandas as pd
import requests
import selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from bs4 import BeautifulSoup
from time import sleep 
import re
import sys 
import os
sys.path.append(os.path.abspath("../src"))   
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns


import funciones as f

Sacar lista supers

In [2]:
url_super = "https://super.facua.org/"
res_facua = requests.get(url_super)
sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})
lista_supers= [mercado.getText().lower() for mercado in lista_mercados]
lista_supers.pop()
lista_supers


['mercadona', 'alcampo', 'carrefour', 'hipercor', 'dia', 'eroski']

Sacar lista productos

In [3]:
url_mercados = f"https://super.facua.org/mercadona/"
res_producto = requests.get(url_mercados)
sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
lista_productos= [mercado.getText().lower().replace(" ", "-") for mercado in lista_prod]
for i, producto in enumerate(lista_productos,start=1):
    id_prod= i
    nombre_producto= producto
    print(nombre_producto)
    print(id_prod)


aceite-de-girasol
1
aceite-de-oliva
2
leche
3


Sacar lista variantes


In [4]:
url_variante = "https://super.facua.org/mercadona/aceite-de-girasol/"
res_variante= requests.get(url_variante)
sopa_variante = BeautifulSoup(res_variante.content, "html.parser")
lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
for i, variante in enumerate(lista_variantes, start=1):
    id_var= i
    nombre_variante= variante
    print(id_var)
    print(nombre_variante)

1
aceite-de-girasol-refinado-02-hacendado-1-l
2
aceite-de-girasol-refinado-02-hacendado-5-l


Lo comento para que no me moleste

In [5]:
# for mercado in lista_supers:
#     df_tablas= pd.DataFrame()
#     driver = webdriver.Chrome()
#     url_mercados = f"https://super.facua.org/{mercado}/"
#     driver.get(url_mercados)
#     sleep(3)
#     #aceptar cookie
#     driver.find_element("css selector", "#rcc-confirm-button").click()
#     #hacer aquí la lista de productos
#     res_producto = requests.get(url_mercados)
#     sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
#     lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
#     lista_productos= [prod.getText().lower().replace(" ", "-") for prod in lista_prod]

#     for i, producto in enumerate(lista_productos):
#         driver.execute_script("window.scrollBy(0, 200);")
#         sleep(2)
#         #selecciona producto
#         driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({i+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#         sopa_variante = BeautifulSoup(driver.page_source, "html.parser")
#         lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
#         lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
#         lista_variantes_bien=[]
#         for elemento in lista_variantes:
#             if "aceite" in elemento or "leche" in elemento:
#                 lista_variantes_bien.append(elemento)
#         for j ,variante in enumerate(lista_variantes_bien):
#             while True:
#                 #selecciona variante
#                 try:
#                     sleep(2)
#                     driver.find_element("css selector", f"body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child({j+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#                     break
#                 except:
#                     driver.execute_script("window.scrollBy(0, 200);")
#             sopa_mercados = BeautifulSoup(driver.page_source, "html.parser")
#             tabla_general= sopa_mercados.findAll("table")
#             tablaindiv=tabla_general[-1]
#             #por aquí crear las columnas necesarias usando lista de productos y variante (mercado se pone formateado tal cual)
#             df_tablas=pd.concat([df_tablas, f.sacar_tabla(tablaindiv,mercado,producto,variante)])
#             driver.back()
#         driver.back()    
#     driver.close()
#     df_tablas.to_csv(f'../datos/datos{mercado}.csv', index=False)


In [6]:
# df_tablas.head()

In [7]:
lista=[]
lista_variantes_bien=["aceite de oliva", "oliva de aceite", "oliva aceite de", "leche merengada azul", "merengada leche azul", "merengada zaul leche", "caca"]     
for elemento in lista_variantes_bien:
    if "aceite" in elemento or "leche" in elemento:
        lista.append(elemento)
lista

['aceite de oliva',
 'oliva de aceite',
 'oliva aceite de',
 'leche merengada azul',
 'merengada leche azul',
 'merengada zaul leche']

Carga de datos

In [8]:
alcampo=pd.read_csv("../datos/datosalcampo.csv")
mercadona=pd.read_csv("../datos/datosmercadona.csv")
carrefour=pd.read_csv("../datos/datoscarrefour.csv")
dia=pd.read_csv("../datos/datosdia.csv")
eroski=pd.read_csv("../datos/datoseroski.csv")
hipercor=pd.read_csv("../datos/datoshipercor.csv")

In [9]:
alcampo.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,30/06/2024,"5,88",=,alcampo,aceite-de-girasol,campomar-nature-aceite-de-girasol-ecológico-ca...
1,01/07/2024,"5,88",=,alcampo,aceite-de-girasol,campomar-nature-aceite-de-girasol-ecológico-ca...
2,12/07/2024,"5,88",=,alcampo,aceite-de-girasol,campomar-nature-aceite-de-girasol-ecológico-ca...
3,13/07/2024,"5,88",=,alcampo,aceite-de-girasol,campomar-nature-aceite-de-girasol-ecológico-ca...
4,14/07/2024,"5,88",=,alcampo,aceite-de-girasol,campomar-nature-aceite-de-girasol-ecológico-ca...


In [10]:
mercadona.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
1,13/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
2,14/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
3,15/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
4,16/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l


In [11]:
eroski.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
1,13/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
2,14/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
3,15/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
4,16/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro


In [12]:
dia.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,29/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
1,30/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
2,31/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
3,01/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
4,02/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...


In [13]:
hipercor.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,13/07/2024,"1,40",=,hipercor,aceite-de-girasol,abrilsol-aceite-refinado-de-girasol-botella-1-l
1,14/07/2024,"1,40",=,hipercor,aceite-de-girasol,abrilsol-aceite-refinado-de-girasol-botella-1-l
2,15/07/2024,"1,40",=,hipercor,aceite-de-girasol,abrilsol-aceite-refinado-de-girasol-botella-1-l
3,16/07/2024,"1,40",=,hipercor,aceite-de-girasol,abrilsol-aceite-refinado-de-girasol-botella-1-l
4,17/07/2024,"1,40",=,hipercor,aceite-de-girasol,abrilsol-aceite-refinado-de-girasol-botella-1-l


In [14]:
carrefour.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


Creación Data Frame Total y Exploración inicial

In [15]:
df= pd.DataFrame()
df_total= pd.concat([df,carrefour,mercadona,eroski,dia,hipercor,alcampo])

In [16]:
df_total.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


**RESULTADOS INICIALES DE EXPLORACION**    


Contamos con un Data Frame de 134114 filas y 6 columnas.

*Nulos:*  

- No cuenta con ningun dato nulo que tengamos que tratar


*Duplicados:*  

- Contamos con 1160 filas duplicadas lo que supone un 0,86% del total de filas.  
- Tras analizarlos y comprobar la web, nos dimos cuenta de que los podemos eliminar ya que en la propia web aparecen productos duplicados(Nos quedamos con un Data Frame con 132954 filas y 6 columnas). 

*Formatos:*  

- En la columna variacion quise convertir los iguales a 0, quedarme solo con los porcentajes y luego ya pasar a float.  
- En la columna precio cambié las , por los . y los pasé a float.   
- En la columna Día la fecha debería estar en formato fecha así que la cambié. 





Tratamos los duplicados

In [17]:
df_total.shape

(134114, 6)

In [18]:
df_total.duplicated().sum()

np.int64(1160)

In [19]:
round((df_total.duplicated().sum()/len(df_total))*100, 2)

np.float64(0.86)

In [20]:
df_total[df_total.duplicated()].head()

,Día,Precio (€),Variación,supermercado,producto,variante
11872,15/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11873,16/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11874,17/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11875,18/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11876,19/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...


In [21]:
df_total.drop_duplicates(inplace=True)

Formateamos

In [22]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132954 entries, 0 to 32867
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Día           132954 non-null  object
 1   Precio (€)    132954 non-null  object
 2   Variación     132954 non-null  object
 3   supermercado  132954 non-null  object
 4   producto      132954 non-null  object
 5   variante      132954 non-null  object
dtypes: object(6)
memory usage: 7.1+ MB


In [23]:
df_total["Precio (€)"]=df_total["Precio (€)"].str.replace(",", ".").astype(float)



In [24]:
df_total.sample(5)

,Día,Precio (€),Variación,supermercado,producto,variante
3221,25/10/2024,8.99,=,eroski,aceite-de-oliva,aceite-de-oliva-1-urzante-botella-1-litro
23398,07/10/2024,7.74,=,alcampo,leche,kaiku-leche-semidesnatada-de-vaca-sin-lactosa-...
6547,09/08/2024,0.99,=,dia,leche,leche-desnatada-rica-en-proteínas-puleva-pack-...
10021,04/08/2024,8.00,=,hipercor,aceite-de-oliva,el-corte-ingles-aceite-de-oliva-suave-04-conti...
19236,27/07/2024,4.50,=,alcampo,leche,central-lechera-asturiana-leche-de-vaca-entera...


In [25]:
df_total["Variación"]=df_total["Variación"].str.replace("+","").str.replace(",", ".").str.split(" ").str[0].str.replace("=", "0").astype(float)

In [109]:
df_total["Día"]=pd.to_datetime(df_total["Día"], dayfirst= True)

In [110]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132954 entries, 0 to 32867
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Día           132954 non-null  datetime64[ns]
 1   Precio (€)    132954 non-null  float64       
 2   Variación     132954 non-null  float64       
 3   supermercado  132954 non-null  object        
 4   producto      132954 non-null  object        
 5   variante      132954 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 7.1+ MB


In [108]:
df_total.to_csv(f'../datos/datos_Supers_completo.csv', index=False)